In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data

# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Found 100 validated image filenames.


In [ ]:
#####PRETRAINED MODEL

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, stride=1, use_projection=False):
    shortcut = x
    if use_projection:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # First convolutional layer
    x = Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Add shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def ResNet50(input_shape=(227, 227, 3), num_classes=5):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64, use_projection=True)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)

    x = residual_block(x, filters=128, stride=2, use_projection=True)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)

    x = residual_block(x, filters=256, stride=2, use_projection=True)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)

    x = residual_block(x, filters=512, stride=2, use_projection=True)
    x = residual_block(x, filters=512)
    x = residual_block(x, filters=512)

    # Final layers
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create ResNet50 model
resnet50_model = ResNet50()

# Display the model summary
resnet50_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
==================================================================================================
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 114, 114, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 114, 114, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                                 
                                                                                                  
 activation (Activation)        (None, 114, 114, 64  0           ['batch_normalization[0][0]']    
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 57, 57, 64)   0           ['activation[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)              (None, 57, 57, 64)   36928       ['max_pooling2d[0][0]']          
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_2[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_1 (Activation)      (None, 57, 57, 64)   0           ['batch_normalization_2[0][0]']  
                                                                                                  
 conv2d_3 (Conv2D)              (None, 57, 57, 64)   36928       ['activation_1[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)              (None, 57, 57, 64)   4160        ['max_pooling2d[0][0]']          
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_1[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 add (Add)                      (None, 57, 57, 64)   0           ['batch_normalization_3[0][0]',  
                                                                  'batch_normalization_1[0][0]']  
                                                                                                  
 activation_2 (Activation)      (None, 57, 57, 64)   0           ['add[0][0]']                    
                                                                                                  
 conv2d_4 (Conv2D)              (None, 57, 57, 64)   36928       ['activation_2[0][0]']           
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_4[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 57, 57, 64)   0           ['batch_normalization_4[0][0]']  
                                                                                                  
 conv2d_5 (Conv2D)              (None, 57, 57, 64)   36928       ['activation_3[0][0]']           
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 add_1 (Add)                    (None, 57, 57, 64)   0           ['batch_normalization_5[0][0]',  
                                                                  'activation_2[0][0]']           
                                                                                                  
 activation_4 (Activation)      (None, 57, 57, 64)   0           ['add_1[0][0]']                  
                                                                                                  
 conv2d_6 (Conv2D)              (None, 57, 57, 64)   36928       ['activation_4[0][0]']           
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_6[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_5 (Activation)      (None, 57, 57, 64)   0           ['batch_normalization_6[0][0]']  
                                                                                                  
 conv2d_7 (Conv2D)              (None, 57, 57, 64)   36928       ['activation_5[0][0]']           
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 57, 57, 64)  256         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 add_2 (Add)                    (None, 57, 57, 64)   0           ['batch_normalization_7[0][0]',  
                                                                  'activation_4[0][0]']           
                                                                                                  
 activation_6 (Activation)      (None, 57, 57, 64)   0           ['add_2[0][0]']                  
                                                                                                  
 conv2d_9 (Conv2D)              (None, 29, 29, 128)  73856       ['activation_6[0][0]']           
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 29, 29, 128)  512        ['conv2d_9[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_7 (Activation)      (None, 29, 29, 128)  0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_10 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_7[0][0]']           
                                                                                                  
 conv2d_8 (Conv2D)              (None, 29, 29, 128)  8320        ['activation_6[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 29, 29, 128)  512        ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 29, 29, 128)  512        ['conv2d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 add_3 (Add)                    (None, 29, 29, 128)  0           ['batch_normalization_10[0][0]', 
                                                                  'batch_normalization_8[0][0]']  
                                                                                                  
 activation_8 (Activation)      (None, 29, 29, 128)  0           ['add_3[0][0]']                  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_8[0][0]']           
                                                                                                  
 batch_normalization_11 (BatchN  (None, 29, 29, 128)  512        ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_9 (Activation)      (None, 29, 29, 128)  0           ['batch_normalization_11[0][0]'] 
                                                                                                  
 conv2d_12 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_9[0][0]']           
                                                                                                  
 batch_normalization_12 (BatchN  (None, 29, 29, 128)  512        ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_4 (Add)                    (None, 29, 29, 128)  0           ['batch_normalization_12[0][0]', 
                                                                  'activation_8[0][0]']           
                                                                                                  
 activation_10 (Activation)     (None, 29, 29, 128)  0           ['add_4[0][0]']                  
                                                                                                  
 conv2d_13 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_10[0][0]']          
                                                                                                  
 batch_normalization_13 (BatchN  (None, 29, 29, 128)  512        ['conv2d_13[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_11 (Activation)     (None, 29, 29, 128)  0           ['batch_normalization_13[0][0]'] 
                                                                                                  
 conv2d_14 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_11[0][0]']          
                                                                                                  
 batch_normalization_14 (BatchN  (None, 29, 29, 128)  512        ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_5 (Add)                    (None, 29, 29, 128)  0           ['batch_normalization_14[0][0]', 
                                                                  'activation_10[0][0]']          
                                                                                                  
 activation_12 (Activation)     (None, 29, 29, 128)  0           ['add_5[0][0]']                  
                                                                                                  
 conv2d_15 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_12[0][0]']          
                                                                                                  
 batch_normalization_15 (BatchN  (None, 29, 29, 128)  512        ['conv2d_15[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_13 (Activation)     (None, 29, 29, 128)  0           ['batch_normalization_15[0][0]'] 
                                                                                                  
 conv2d_16 (Conv2D)             (None, 29, 29, 128)  147584      ['activation_13[0][0]']          
                                                                                                  
 batch_normalization_16 (BatchN  (None, 29, 29, 128)  512        ['conv2d_16[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_6 (Add)                    (None, 29, 29, 128)  0           ['batch_normalization_16[0][0]', 
                                                                  'activation_12[0][0]']          
                                                                                                  
 activation_14 (Activation)     (None, 29, 29, 128)  0           ['add_6[0][0]']                  
                                                                                                  
 conv2d_18 (Conv2D)             (None, 15, 15, 256)  295168      ['activation_14[0][0]']          
                                                                                                  
 batch_normalization_18 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_15 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_18[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_15[0][0]']          
                                                                                                  
 conv2d_17 (Conv2D)             (None, 15, 15, 256)  33024       ['activation_14[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_17 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_17[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_7 (Add)                    (None, 15, 15, 256)  0           ['batch_normalization_19[0][0]', 
                                                                  'batch_normalization_17[0][0]'] 
                                                                                                  
 activation_16 (Activation)     (None, 15, 15, 256)  0           ['add_7[0][0]']                  
                                                                                                  
 conv2d_20 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_16[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_20[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_17 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_20[0][0]'] 
                                                                                                  
 conv2d_21 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_17[0][0]']          
                                                                                                  
 batch_normalization_21 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_8 (Add)                    (None, 15, 15, 256)  0           ['batch_normalization_21[0][0]', 
                                                                  'activation_16[0][0]']          
                                                                                                  
 activation_18 (Activation)     (None, 15, 15, 256)  0           ['add_8[0][0]']                  
                                                                                                  
 conv2d_22 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_18[0][0]']          
                                                                                                  
 batch_normalization_22 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_19 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_23 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_19[0][0]']          
                                                                                                  
 batch_normalization_23 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_23[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_9 (Add)                    (None, 15, 15, 256)  0           ['batch_normalization_23[0][0]', 
                                                                  'activation_18[0][0]']          
                                                                                                  
 activation_20 (Activation)     (None, 15, 15, 256)  0           ['add_9[0][0]']                  
                                                                                                  
 conv2d_24 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_20[0][0]']          
                                                                                                  
 batch_normalization_24 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_24[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_25 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_25[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_10 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_25[0][0]', 
                                                                  'activation_20[0][0]']          
                                                                                                  
 activation_22 (Activation)     (None, 15, 15, 256)  0           ['add_10[0][0]']                 
                                                                                                  
 conv2d_26 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_22[0][0]']          
                                                                                                  
 batch_normalization_26 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_23 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 conv2d_27 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_23[0][0]']          
                                                                                                  
 batch_normalization_27 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_27[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_11 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_27[0][0]', 
                                                                  'activation_22[0][0]']          
                                                                                                  
 activation_24 (Activation)     (None, 15, 15, 256)  0           ['add_11[0][0]']                 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_24[0][0]']          
                                                                                                  
 batch_normalization_28 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_28[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_25 (Activation)     (None, 15, 15, 256)  0           ['batch_normalization_28[0][0]'] 
                                                                                                  
 conv2d_29 (Conv2D)             (None, 15, 15, 256)  590080      ['activation_25[0][0]']          
                                                                                                  
 batch_normalization_29 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_12 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_29[0][0]', 
                                                                  'activation_24[0][0]']          
                                                                                                  
 activation_26 (Activation)     (None, 15, 15, 256)  0           ['add_12[0][0]']                 
                                                                                                  
 conv2d_31 (Conv2D)             (None, 8, 8, 512)    1180160     ['activation_26[0][0]']          
                                                                                                  
 batch_normalization_31 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_31[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_27 (Activation)     (None, 8, 8, 512)    0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 conv2d_32 (Conv2D)             (None, 8, 8, 512)    2359808     ['activation_27[0][0]']          
                                                                                                  
 conv2d_30 (Conv2D)             (None, 8, 8, 512)    131584      ['activation_26[0][0]']          
                                                                                                  
 batch_normalization_32 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_32[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_30 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_30[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_13 (Add)                   (None, 8, 8, 512)    0           ['batch_normalization_32[0][0]', 
                                                                  'batch_normalization_30[0][0]'] 
                                                                                                  
 activation_28 (Activation)     (None, 8, 8, 512)    0           ['add_13[0][0]']                 
                                                                                                  
 conv2d_33 (Conv2D)             (None, 8, 8, 512)    2359808     ['activation_28[0][0]']          
                                                                                                  
 batch_normalization_33 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_29 (Activation)     (None, 8, 8, 512)    0           ['batch_normalization_33[0][0]'] 
                                                                                                  
 conv2d_34 (Conv2D)             (None, 8, 8, 512)    2359808     ['activation_29[0][0]']          
                                                                                                  
 batch_normalization_34 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_14 (Add)                   (None, 8, 8, 512)    0           ['batch_normalization_34[0][0]', 
                                                                  'activation_28[0][0]']          
                                                                                                  
 activation_30 (Activation)     (None, 8, 8, 512)    0           ['add_14[0][0]']                 
                                                                                                  
 conv2d_35 (Conv2D)             (None, 8, 8, 512)    2359808     ['activation_30[0][0]']          
                                                                                                  
 batch_normalization_35 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_35[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_31 (Activation)     (None, 8, 8, 512)    0           ['batch_normalization_35[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 8, 8, 512)    2359808     ['activation_31[0][0]']          
                                                                                                  
 batch_normalization_36 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_15 (Add)                   (None, 8, 8, 512)    0           ['batch_normalization_36[0][0]', 
                                                                  'activation_30[0][0]']          
                                                                                                  
 activation_32 (Activation)     (None, 8, 8, 512)    0           ['add_15[0][0]']                 
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 512)   0           ['activation_32[0][0]']          
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 512)          0           ['average_pooling2d[0][0]']      
                                                                                                  
 dense (Dense)                  (None, 5)            2565        ['flatten[0][0]']                
                                                                                                  
==================================================================================================
Total params: 21,317,189
Trainable params: 21,300,037
Non-trainable params: 17,152
__________________________________________________________________________________________________


In [ ]:

resnet50_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
               
resnet50_model.fit(train_generator, 
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
   ) 


####output
Epoch 1/40
248/248 [==============================] - 95s 342ms/step - loss: 1.3395 - accuracy: 0.4865 - val_loss: 2.7333 - val_accuracy: 0.2061
Epoch 2/40
248/248 [==============================] - 80s 324ms/step - loss: 1.0810 - accuracy: 0.5756 - val_loss: 1.6324 - val_accuracy: 0.3833
Epoch 3/40
248/248 [==============================] - 77s 311ms/step - loss: 1.0123 - accuracy: 0.6034 - val_loss: 1.3618 - val_accuracy: 0.4611
Epoch 4/40
248/248 [==============================] - 80s 320ms/step - loss: 0.9654 - accuracy: 0.6221 - val_loss: 1.5169 - val_accuracy: 0.4025
Epoch 5/40
248/248 [==============================] - 79s 319ms/step - loss: 0.9078 - accuracy: 0.6484 - val_loss: 1.2966 - val_accuracy: 0.4652
Epoch 6/40
248/248 [==============================] - 78s 314ms/step - loss: 0.9135 - accuracy: 0.6441 - val_loss: 1.4587 - val_accuracy: 0.4505
Epoch 7/40
248/248 [==============================] - 71s 285ms/step - loss: 0.8455 - accuracy: 0.6734 - val_loss: 1.1951 - val_accuracy: 0.5333
Epoch 8/40
248/248 [==============================] - 81s 326ms/step - loss: 0.8332 - accuracy: 0.6843 - val_loss: 1.1367 - val_accuracy: 0.5591
Epoch 9/40
248/248 [==============================] - 73s 295ms/step - loss: 0.8010 - accuracy: 0.6994 - val_loss: 1.2691 - val_accuracy: 0.5364
Epoch 10/40
248/248 [==============================] - 73s 295ms/step - loss: 0.7842 - accuracy: 0.7081 - val_loss: 1.2890 - val_accuracy: 0.5045
Epoch 11/40
248/248 [==============================] - 71s 287ms/step - loss: 0.8036 - accuracy: 0.6934 - val_loss: 0.9537 - val_accuracy: 0.6571
Epoch 12/40
248/248 [==============================] - 74s 298ms/step - loss: 0.7489 - accuracy: 0.7178 - val_loss: 0.9610 - val_accuracy: 0.6222
Epoch 13/40
248/248 [==============================] - 71s 284ms/step - loss: 0.7406 - accuracy: 0.7232 - val_loss: 1.8443 - val_accuracy: 0.4990
Epoch 14/40
248/248 [==============================] - 73s 293ms/step - loss: 0.7087 - accuracy: 0.7348 - val_loss: 1.1407 - val_accuracy: 0.6035
Epoch 15/40
248/248 [==============================] - 72s 291ms/step - loss: 0.6940 - accuracy: 0.7399 - val_loss: 1.6511 - val_accuracy: 0.4298
Epoch 16/40
248/248 [==============================] - 74s 299ms/step - loss: 0.6822 - accuracy: 0.7423 - val_loss: 0.9037 - val_accuracy: 0.6591
Epoch 17/40
248/248 [==============================] - 71s 285ms/step - loss: 0.6816 - accuracy: 0.7446 - val_loss: 1.8166 - val_accuracy: 0.4909
Epoch 18/40
248/248 [==============================] - 74s 296ms/step - loss: 0.6692 - accuracy: 0.7491 - val_loss: 0.8350 - val_accuracy: 0.7015
Epoch 19/40
248/248 [==============================] - 71s 287ms/step - loss: 0.6396 - accuracy: 0.7605 - val_loss: 1.0367 - val_accuracy: 0.6051
Epoch 20/40
248/248 [==============================] - 77s 309ms/step - loss: 0.6412 - accuracy: 0.7611 - val_loss: 1.1777 - val_accuracy: 0.6263
Epoch 21/40
248/248 [==============================] - 72s 288ms/step - loss: 0.6482 - accuracy: 0.7598 - val_loss: 10.3072 - val_accuracy: 0.5268
Epoch 22/40
248/248 [==============================] - 74s 298ms/step - loss: 0.6120 - accuracy: 0.7679 - val_loss: 1.8083 - val_accuracy: 0.6247
Epoch 23/40
248/248 [==============================] - 72s 291ms/step - loss: 0.6006 - accuracy: 0.7754 - val_loss: 1.1416 - val_accuracy: 0.6237
Epoch 24/40
248/248 [==============================] - 79s 317ms/step - loss: 0.5943 - accuracy: 0.7785 - val_loss: 2.1000 - val_accuracy: 0.4323
Epoch 25/40
248/248 [==============================] - 71s 284ms/step - loss: 0.5797 - accuracy: 0.7860 - val_loss: 0.9031 - val_accuracy: 0.6727
Epoch 26/40
248/248 [==============================] - 73s 295ms/step - loss: 0.5711 - accuracy: 0.7843 - val_loss: 1.1866 - val_accuracy: 0.5803
Epoch 27/40
248/248 [==============================] - 72s 291ms/step - loss: 0.5482 - accuracy: 0.7994 - val_loss: 1.1587 - val_accuracy: 0.6182
Epoch 28/40
248/248 [==============================] - 74s 296ms/step - loss: 0.5567 - accuracy: 0.8001 - val_loss: 1.0715 - val_accuracy: 0.6141
Epoch 29/40
248/248 [==============================] - 71s 286ms/step - loss: 0.5336 - accuracy: 0.8033 - val_loss: 0.9237 - val_accuracy: 0.6515
Epoch 30/40
248/248 [==============================] - 74s 298ms/step - loss: 0.5157 - accuracy: 0.8128 - val_loss: 0.7923 - val_accuracy: 0.7187
Epoch 31/40
248/248 [==============================] - 78s 312ms/step - loss: 0.5095 - accuracy: 0.8135 - val_loss: 1.2877 - val_accuracy: 0.5566
Epoch 32/40
248/248 [==============================] - 78s 314ms/step - loss: 0.4935 - accuracy: 0.8203 - val_loss: 1.0260 - val_accuracy: 0.6404
Epoch 33/40
248/248 [==============================] - 70s 283ms/step - loss: 0.4944 - accuracy: 0.8187 - val_loss: 1.0167 - val_accuracy: 0.6485
Epoch 34/40
248/248 [==============================] - 72s 288ms/step - loss: 0.4812 - accuracy: 0.8277 - val_loss: 1.0302 - val_accuracy: 0.6399
Epoch 35/40
248/248 [==============================] - 71s 287ms/step - loss: 0.4611 - accuracy: 0.8311 - val_loss: 0.9923 - val_accuracy: 0.6621
Epoch 36/40
248/248 [==============================] - 73s 292ms/step - loss: 0.4513 - accuracy: 0.8298 - val_loss: 1.2575 - val_accuracy: 0.6035
Epoch 37/40
248/248 [==============================] - 69s 279ms/step - loss: 0.4471 - accuracy: 0.8369 - val_loss: 0.9494 - val_accuracy: 0.6682
Epoch 38/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4359 - accuracy: 0.8405 - val_loss: 0.8327 - val_accuracy: 0.7545
Epoch 39/40
248/248 [==============================] - 70s 282ms/step - loss: 0.4237 - accuracy: 0.8489 - val_loss: 1.1590 - val_accuracy: 0.6389
Epoch 40/40
248/248 [==============================] - 73s 294ms/step - loss: 0.4163 - accuracy: 0.8485 - val_loss: 0.8049 - val_accuracy: 0.7152


In [3]:
from tensorflow.keras import layers, Model, Input

def residual_block(x, filters, stride=1, use_projection=False):
    shortcut = x
    if use_projection:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add dropout for regularization
    x = layers.Dropout(0.3)(x)

    # Add shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def ResNet50_modified(input_shape=(224, 224, 3), num_classes=5):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    # Reduced number of residual blocks for brevity
    x = residual_block(x, filters=64, use_projection=True)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, stride=2, use_projection=True)
    x = residual_block(x, filters=128)

    # Fewer residual blocks for illustration purposes
    x = residual_block(x, filters=256, stride=2, use_projection=True)
    x = residual_block(x, filters=256)

    x = layers.AveragePooling2D(pool_size=(7, 7))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the modified ResNet50 model
modified_resnet50_model = ResNet50_modified()

# Display the model summary
modified_resnet50_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [4]:

modified_resnet50_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
               
modified_resnet50_model.fit(train_generator, 
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
   ) 


Epoch 1/40
248/248 [==============================] - 87s 315ms/step - loss: 1.2526 - accuracy: 0.5020 - val_loss: 1.7502 - val_accuracy: 0.3470
Epoch 2/40
248/248 [==============================] - 77s 305ms/step - loss: 1.0527 - accuracy: 0.5784 - val_loss: 1.4090 - val_accuracy: 0.4455
Epoch 3/40
248/248 [==============================] - 72s 287ms/step - loss: 0.9792 - accuracy: 0.6187 - val_loss: 1.9027 - val_accuracy: 0.4146
Epoch 4/40
248/248 [==============================] - 70s 276ms/step - loss: 0.9164 - accuracy: 0.6439 - val_loss: 1.4428 - val_accuracy: 0.5091
Epoch 5/40
248/248 [==============================] - 73s 289ms/step - loss: 0.8915 - accuracy: 0.6510 - val_loss: 1.3247 - val_accuracy: 0.4833
Epoch 6/40
248/248 [==============================] - 73s 288ms/step - loss: 0.8607 - accuracy: 0.6736 - val_loss: 1.6130 - val_accuracy: 0.5242
Epoch 7/40
248/248 [==============================] - 72s 285ms/step - loss: 0.8112 - accuracy: 0.6912 - val_loss: 1.4683 - val_ac